<a href="https://colab.research.google.com/github/Rybus07/space-legends-data/blob/feature%2F-pagination-sample-script/DSCI_511_project_test_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Testing out filtering API calls by year, based on [FAQ](https://github.com/TheSpaceDevs/Tutorials/blob/main/tutorials/getting_started_LL2/README.md#querying--filtering-past-launches-in-python):

In [14]:
from datetime import datetime, timedelta # added timedelta for pause message
from zoneinfo import ZoneInfo # for local Philadelphia timezone
import dateutil.parser as dateparser
import pprint # changed to import the whole module
import requests
import time # for sleep timer
import json # for saving the file at the end

In [2]:
start_time = dateparser.parse("1/1/25")
end_time = dateparser.parse("2/28/25")

print("Start date:", start_time.isoformat())
print("End date:", end_time.isoformat())

Start date: 2025-01-01T00:00:00
End date: 2025-02-28T00:00:00


For this sample run of pagination, I streamlined around the date filters.

Plan on adding them back for specific date ranges.

In [15]:
#Set the filter parameters with the start and end date:
#net_filters = f'net__gte={start_time.isoformat()}&net__lte={end_time.isoformat()}'

mode = 'mode=detailed' #setting this mode to detailed returns all related objects
limit = 'limit=100' #this is the max!

ordering = 'ordering=net' #orders in ascending date order, I think

#Assemble the full URL for the query:
current_url = "https://ll.thespacedevs.com/2.3.0/launches/previous/" + "?" + "&".join(
    (mode, limit, ordering)
)

print(f'Query URL: {current_url}')

Query URL: https://ll.thespacedevs.com/2.3.0/launches/previous/?mode=detailed&limit=100&ordering=net


Used the following link for inspiration:

https://stackoverflow.com/questions/56206038/how-to-loop-through-paginated-api-using-python




In [16]:
###Pagination Loop for first 9 pagination calls using while loop
### Added timer module for pause
## This is configure for test run:
## Makes 4 pagination calls, pauses for 3 min, makes 5 more calls

# Create an empty list to store pagination launches
sample_page_launches = []

# Create a counter for each API call
api_call_count = 0

# Start the loop that runs as long as current_url is not None
# If the API call fails, then current_url is set to None
## Added < 9 for the first nine calls (test-run paginations)
print("STARTING DATA ACQUISITION")
while current_url and api_call_count < 9:

    # Output which page we're on
    print(f"Request #: {api_call_count + 1}")
    # Make the API call
    response = requests.get(current_url)

    # status code of 200 means it was successful
    # continues if successful
    if response.status_code == 200:

        # Parse the JSON response into a dictionary
        raw_data = response.json()

        # launch records are inside the 'results' key
        # Add the new launches from the dictionary to our sample list
        sample_page_launches.extend(raw_data['results'])

        # THIS IS THE PAGINATION: Get the URL for the next page.
        # If it's the last page, data['next'] will be 'None'
        # and the 'while' loop will stop.
        current_url = raw_data['next']

        print(f"Success! So far collected {len(sample_page_launches)} total launches.")

        api_call_count += 1 # Increment counter for each API call

        ### Add timer logic to pause after certain number of API calls
        ### For full pagination: Pause after 14 calls for 1hr
        # Check if the counter is a multiple of 14 (api_call_count % 14 == 0)
        # Using == 4 for test run
        if api_call_count == 4 and current_url is not None:

            # Get local Philadelphia time for pause message
            philly_tz = ZoneInfo('America/New_York')
            time_in_philly = datetime.now(philly_tz)
            length_of_pause = timedelta(minutes=3) # set for 3 min for testing
            resume_time = time_in_philly + length_of_pause

            # Output PAUSE message with local Philadelphia time API calls resume
            print(f"{api_call_count} CALLS MADE. PAUSING FOR {length_of_pause} MIN.")
            print(f"Resuming at {resume_time.strftime('%I:%M:%S %p %Z')}")

            # Full Pagination: Set sleep timer (3601) for 1hr and 1 sec
            # Test run: Set sleep timer (180) for 3 min
            time.sleep(180)

            # Output RESUME message
            print("RESUMING API CALLS")
    else:
        # If it fails, prints an error and stop
        print(f"Error! Status code: {response.status_code}")
        current_url = None # Stop the loop

print(f"FINISHED: Collected {len(sample_page_launches)} total launches!!!")


STARTING DATA ACQUISITION
Request #: 1
Success! So far collected 100 total launches.
Request #: 2
Success! So far collected 200 total launches.
Request #: 3
Success! So far collected 300 total launches.
Request #: 4
Success! So far collected 400 total launches.
4 CALLS MADE. PAUSING FOR 0:03:00 MIN.
Resuming at 05:01:28 PM EST
RESUMING API CALLS
Request #: 5
Success! So far collected 500 total launches.
Request #: 6
Success! So far collected 600 total launches.
Request #: 7
Success! So far collected 700 total launches.
Request #: 8
Success! So far collected 800 total launches.
Request #: 9
Success! So far collected 900 total launches.
FINISHED: Collected 900 total launches!!!


In [17]:
### Save the final data
print(f"Saving {len(sample_page_launches)} launches to a file...")

# 'with open' handles closing the file automatically
with open('raw_baseline_launches.json', 'w', encoding='utf-8') as f:
    # json.dump writes the list to the file
    # indent=4 makes it human-readable
    json.dump(sample_page_launches, f, indent=4)

print("Save complete!")

Saving 900 launches to a file...
Save complete!


In [ ]:
"""
#This is the actual API call!
response = requests.get(url)

print(response.status_code) #status code of 200 means it was
"""

200


Found some code using 'indent' in pprint to make nested list a little more readble.

https://realpython.com/python-pretty-print/

Let me know what you all think.

In [18]:


#Look at a sample of the data we received
"""
#raw_data = response.json()

#launch_sample = raw_data['results'] #launch records are inside the 'results' key
"""
# Created an PrettyPrinter object, configured for indent
# btw default indent = 1
# Downside to creating pp object: if you want to use pprint it is now 'pp.print'
pp = pprint.PrettyPrinter(indent=4)

print(f"Collected a total of {len(sample_page_launches)} launches")
print("The first 10 look like:")

# Called pprint using pp object to output with modified indent
pp.pprint(sample_page_launches[:10])

Collected a total of 900 launches
The first 10 look like:
[   {   'agency_launch_attempt_count': 1,
        'agency_launch_attempt_count_year': 1,
        'failreason': '',
        'flightclub_url': None,
        'hashtag': None,
        'id': 'e3df2ecd-c239-472f-95e4-2b89b4f75800',
        'image': {   'credit': None,
                     'id': 1844,
                     'image_url': 'https://thespacedevs-prod.nyc3.digitaloceanspaces.com/media/images/sputnik_8k74ps_image_20210830185541.jpg',
                     'license': {   'id': 1,
                                    'link': None,
                                    'name': 'Unknown',
                                    'priority': 9},
                     'name': '[AUTO] Sputnik 8K74PS - image',
                     'single_use': True,
                     'thumbnail_url': 'https://thespacedevs-prod.nyc3.digitaloceanspaces.com/media/images/255bauto255d__image_thumbnail_20240305193923.jpeg',
                     'variants': []},
 

This lookup date tool is great. It appears the first launches recorded in Space Devs were in 1957.

In [11]:
#Look at the dates for each launch in data we received
for item in sample_page_launches[:10]:
    print(item["net"])

1970-09-21T07:43:00Z
1970-09-22T13:00:01Z
1970-09-25T05:00:00Z
1970-09-25T14:05:00Z
1970-09-29T08:14:01Z
1970-10-01T08:20:03Z
1970-10-03T10:26:45Z
1970-10-08T12:39:02Z
1970-10-08T15:10:03Z
1970-10-09T11:04:58Z


Check on API throttle, based on [docs](https://ll.thespacedevs.com/2.3.0/api-throttle/)

In [19]:
#Check on API throttle
API_throttle_URL = "https://ll.thespacedevs.com/2.3.0/api-throttle/"

response_throttle = requests.get(API_throttle_URL)

In [20]:
throttle_data = response_throttle.json()
# Had to implement the pp object to view pprint
pp.pprint(throttle_data)

{   'current_use': 9,
    'ident': '35.227.137.89',
    'limit_frequency_secs': 3600,
    'next_use_secs': 0,
    'your_request_limit': 15}


Test out automatically dividing an input year into months

In [ ]:
year = 2014
year_month_starts = [dateparser.parse("1/1/"+str(year)), dateparser.parse("2/1/"+str(year)), dateparser.parse("3/1/"+str(year)),
                     dateparser.parse("4/1/"+str(year)), dateparser.parse("5/1/"+str(year)), dateparser.parse("6/1/"+str(year)),
                     dateparser.parse("7/1/"+str(year)), dateparser.parse("8/1/"+str(year)), dateparser.parse("9/1/"+str(year)),
                     dateparser.parse("10/1/"+str(year)), dateparser.parse("11/1/"+str(year)), dateparser.parse("12/1/"+str(year)),
                     dateparser.parse("1/1/"+str(year+1))]

#print(year_month_starts[-1])
for index, month in enumerate(year_month_starts[:-1]):
    start_date = month
    end_date = year_month_starts[index + 1]
    print(f"From {start_date} to {end_date}")

From 2014-01-01 00:00:00 to 2014-02-01 00:00:00
From 2014-02-01 00:00:00 to 2014-03-01 00:00:00
From 2014-03-01 00:00:00 to 2014-04-01 00:00:00
From 2014-04-01 00:00:00 to 2014-05-01 00:00:00
From 2014-05-01 00:00:00 to 2014-06-01 00:00:00
From 2014-06-01 00:00:00 to 2014-07-01 00:00:00
From 2014-07-01 00:00:00 to 2014-08-01 00:00:00
From 2014-08-01 00:00:00 to 2014-09-01 00:00:00
From 2014-09-01 00:00:00 to 2014-10-01 00:00:00
From 2014-10-01 00:00:00 to 2014-11-01 00:00:00
From 2014-11-01 00:00:00 to 2014-12-01 00:00:00
From 2014-12-01 00:00:00 to 2015-01-01 00:00:00
